<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/Word2Vec_Create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install konlpy
!pip3 install jamo

     |████████████████████████████████| 19.4 MB 5.0 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 448 kB 73.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import requests # 파파고 번역 api 사용
import re
import jpype
import requests
from gensim.models import Word2Vec
from jamo import h2j, j2hcj
from konlpy.tag import Okt
import pandas as pd

In [3]:
# 구글 드라이브 연결
import os
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/PMS/"

Mounted at /content/gdrive/


In [7]:
train_data = []

# 분류될 카테고리명
directory_list = ['영어', '외국어', '국어', '문예', '네이밍', '슬로건', '논문', '수학', '과학', '학습•문예', '코딩', '프로그래밍', '해커톤', '메이커톤', '해킹', '보안', '빅데이터', '인공지능', '사물인터넷', 'IT', '컴퓨터', '달리기', '마라톤', '자전거', '수영', '야구', '축구', '농구', '배구', '배드민턴', '테니스', '골프', '볼링', '당구', '탁구 ', '무예', '레포츠', 'e-스포츠', '스포츠', '그림', '그리기', '디자인', '캐릭터', '웹툰', '공예', '만들기', '미술', '동요', '성악', '실용음악', '가요', '국악', '무용', '발레', '댄스', '배우', '오디션', '음악', '무용', '배우', '사진', 'UCC', '동영상', '드론', '요리', '반려동물', '뷰티', '선발', '주산암산', '바둑', '체스', '건축', '제안', '아이디어', '산업', '사회', '창업', '퀴즈', '이색']

data = pd.read_csv(path+"data2.csv", encoding="cp949")
for i in range(len(data)):
  train_data.append(data['Column1'][i])

for i in range(len(directory_list)):
  train_data.append(directory_list[i])
print(len(train_data))
train_data[:10]

7284


['제24회 한뫼이윤재선생 추모 전국 한글사랑 글짓기  그리기 공모전',
 '월드 슈퍼퀸 슈퍼스타 댄스경연대회',
 '언택트 마라톤 제21회 순천남승룡마라톤대회',
 '2022 대한민국최고연설선발중앙회 슬로건 공모전',
 '제2회 한식 콘텐츠 스토리텔링 공모전',
 '제5회 스틸에세이 공모전',
 '제3회 유엔(UN)참전국가 국기 그리기 공모전',
 '제18회 화순전국국악대제전',
 '2021 눈뫼 허웅 선생 추모 ‘한글 사랑 생활 수기’ 공모전',
 '작업현장 안전 데이터 활용 아이디어 공모전']

In [16]:
# 파일 확장자 및 특수문자 제거 후 형태소 명사 단위로 분석

def morpheme(file_name):
  okt = Okt()

  no_extension_file_name = file_name.rsplit('.', 1)[0] 
  cleaned_file_name = re.sub('[-=+,_#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',
                             '', no_extension_file_name)
  cleaned_file_name.upper()

  return cleaned_file_name

In [19]:
cleaning = []
for data in train_data:
  cleaning.append(morpheme(data))

train_data = cleaning
train_data[:10]

['제24회 한뫼이윤재선생 추모 전국 한글사랑 글짓기  그리기 공모전',
 '월드 슈퍼퀸 슈퍼스타 댄스경연대회',
 '언택트 마라톤 제21회 순천남승룡마라톤대회',
 '2022 대한민국최고연설선발중앙회 슬로건 공모전',
 '제2회 한식 콘텐츠 스토리텔링 공모전',
 '제5회 스틸에세이 공모전',
 '제3회 유엔UN참전국가 국기 그리기 공모전',
 '제18회 화순전국국악대제전',
 '2021 눈뫼 허웅 선생 추모 한글 사랑 생활 수기’ 공모전',
 '작업현장 안전 데이터 활용 아이디어 공모전']

In [20]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [23]:
# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()

tokenized_data = []
for sentence in train_data:
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

In [24]:
tokenized_data[:10]

[['제', '24회', '뫼', '이윤재', '선생', '추모', '전국', '한글', '사랑', '글짓기', '그리기', '공모전'],
 ['월드', '슈퍼', '퀸', '슈퍼스타', '댄스', '경연', '대회'],
 ['언택트', '마라톤', '제', '21회', '순천', '남승룡', '마라톤', '대회'],
 ['2022', '대한민국', '최고', '연설', '선발', '중앙회', '슬로건', '공모전'],
 ['제', '2회', '한식', '콘텐츠', '스토리텔링', '공모전'],
 ['제', '5회', '스틸', '에세이', '공모전'],
 ['제', '3회', '유엔', 'UN', '참전', '국가', '국기', '그리기', '공모전'],
 ['제', '18회', '화순', '전국', '국악', '대', '제전'],
 ['2021', '눈뫼', '허웅', '선생', '추모', '한글', '사랑', '생활', '수기', '’', '공모전'],
 ['작업', '현장', '안전', '데이터', '활용', '아이디어', '공모전']]

In [25]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [26]:
model.wv.vectors.shape

(1477, 100)

In [27]:
print(model.wv.most_similar("탁구"))

[('볼링', 0.9998054504394531), ('배드민턴', 0.9997981786727905), ('장', 0.9997916221618652), ('장기', 0.999777615070343), ('대', 0.9997641444206238), ('2019', 0.9997498989105225), ('배구', 0.9997379779815674), ('구', 0.9997300505638123), ('클럽', 0.9997285604476929), ('오픈', 0.9997276067733765)]


In [28]:
model.save(path+"pms.bin")

In [30]:
import gensim

model = gensim.models.Word2Vec.load(path+"ko.bin")
model.wv.save_word2vec_format(path+"ko.bin.gz", binary=False)

In [34]:
from gensim.models import Word2Vec, KeyedVectors

model_2 = Word2Vec(size=200, min_count=1)
model_2.build_vocab(tokenized_data)
total_examples = model_2.corpus_count
print(total_examples)

7284


In [36]:
model_2.build_vocab([list(model.wv.vocab.keys())], update=True)
model_2.intersect_word2vec_format(path+"ko.bin.gz", binary=False)

In [37]:
model_2.train(tokenized_data, total_examples=total_examples, epochs=model_2.iter)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(262685, 271455)

In [38]:
model_2.most_similar("탁구")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('배구', 0.8331664800643921),
 ('배드민턴', 0.7878385782241821),
 ('핸드볼', 0.7865797281265259),
 ('권투', 0.7642523050308228),
 ('테니스', 0.7610983848571777),
 ('양궁', 0.7551153302192688),
 ('스쿼시', 0.7535832524299622),
 ('펜싱', 0.7437219619750977),
 ('농구', 0.7434035539627075),
 ('복싱', 0.7339618802070618)]

In [39]:
len(model_2.wv.vocab)

33713

In [40]:
model_2.save(path+"ko_new.bin")